# W_GL_SEGMENT_DS ETL Process
### Oracle EBS FND_FLEX_VALUES to EDW W_GL_SEGMENT_DS

In [ ]:
%sql
-- 1) PARAMETERS
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID,
  '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID,
  2624634 AS ETL_PROC_WID,
  12345 AS EXECUTION_ID,
  30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Extract flex values
CREATE OR REPLACE TEMP VIEW stg_flex_values AS
SELECT
  fv.CREATION_DATE,
  fvs.LAST_UPDATE_DATE AS LAST_UPDATE_DATE1,
  fv.LAST_UPDATE_DATE AS CHANGED_ON_DT,
  'N' AS DELETE_FLG,
  CAST(fv.FLEX_VALUE_SET_ID AS STRING) AS SEGMENT_LOV_ID,
  COALESCE(
    CASE WHEN fvs.VALIDATION_TYPE = 'I' THEN fv.FLEX_VALUE
         ELSE CONCAT(fv.FLEX_VALUE, '~', fv.PARENT_FLEX_VALUE_LOW)
    END,
    '__UNASSIGNED__'
  ) AS SEGMENT_VAL_CODE,
  CAST(fv.CREATED_BY AS STRING) AS CREATED_BY_ID,
  CAST(fv.LAST_UPDATED_BY AS STRING) AS CHANGED_BY_ID,
  to_timestamp('1899-01-01 00:00:00') AS SRC_EFF_FROM_DT,
  to_timestamp('3714-01-01 00:00:00') AS SRC_EFF_TO_DT,
  CONCAT(CAST(fv.FLEX_VALUE_SET_ID AS STRING), '~', fv.FLEX_VALUE) AS INTEGRATION_ID
FROM workspace.oracle_ebs.FND_FLEX_VALUES fv
INNER JOIN workspace.oracle_ebs.FND_FLEX_VALUE_SETS fvs
  ON fv.FLEX_VALUE_SET_ID = fvs.FLEX_VALUE_SET_ID
CROSS JOIN etl_params p
WHERE fvs.VALIDATION_TYPE IN ('I', 'D')
  AND (fv.LAST_UPDATE_DATE > p.LAST_EXTRACT_DATE OR fvs.LAST_UPDATE_DATE > p.LAST_EXTRACT_DATE);

In [ ]:
%sql
-- 3) INSERT into W_GL_SEGMENT_DS
INSERT INTO workspace.oracle_edw.w_gl_segment_ds
(
  SEGMENT_LOV_ID, SEGMENT_VAL_CODE, CREATED_BY_ID, CHANGED_BY_ID,
  CREATED_ON_DT, CHANGED_ON_DT, AUX1_CHANGED_ON_DT,
  SRC_EFF_FROM_DT, SRC_EFF_TO_DT, DELETE_FLG,
  DATASOURCE_NUM_ID, INTEGRATION_ID, TENANT_ID, X_CUSTOM
)
SELECT
  s.SEGMENT_LOV_ID, s.SEGMENT_VAL_CODE, s.CREATED_BY_ID, s.CHANGED_BY_ID,
  s.CREATION_DATE, s.CHANGED_ON_DT, s.LAST_UPDATE_DATE1,
  s.SRC_EFF_FROM_DT, s.SRC_EFF_TO_DT, s.DELETE_FLG,
  p.DATASOURCE_NUM_ID, s.INTEGRATION_ID, p.TENANT_ID, '0'
FROM stg_flex_values s
CROSS JOIN etl_params p;

In [ ]:
%sql
-- 4) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID AS datasource_num_id,
    'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION' AS package_name,
    'W_GL_SEGMENT_DS' AS target_table_name,
    p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.package_name = src.package_name
  AND tgt.etl_usage_code = src.etl_usage_code
WHEN MATCHED THEN UPDATE SET
  tgt.target_table_name = src.target_table_name, tgt.etl_proc_wid = src.etl_proc_wid,
  tgt.load_plan_id = src.load_plan_id, tgt.wip_load_start_date = src.wip_load_start_date,
  tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT (
  datasource_num_id, package_name, target_table_name, etl_usage_code,
  etl_proc_wid, load_plan_id, wip_load_start_date, last_max_date, etl_load_date, committed
) VALUES (
  src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed
);

In [ ]:
%sql
-- 5) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION', 'W_GL_SEGMENT_DS',
  p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID,
  date_add(current_timestamp(), -p.PRUNE_DAYS), NULL, current_timestamp(),
  CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END
FROM etl_params p;

In [ ]:
%sql
-- 6) CLEANUP
DROP VIEW IF EXISTS stg_flex_values;
DROP VIEW IF EXISTS etl_params;